In [1]:
# ch4-2.py
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(module='sklearn*', action='ignore', category=DeprecationWarning)

In [5]:
# 데이터로드 (NHIS_OPEN_GJ_2017.csv : 데이터 원본 파일)
# encoding : 윈도우즈 환경에서의 한글 처리
# engine : python 3.6에서 한글이 포함된 파일이름 사용
rawData_org = pd.read_csv('datasets/국민건강정보/NHIS_OPEN_GJ_2017.csv', encoding='CP949', engine='python')

In [7]:
# 원본데이터의 튜플 수와 컬럼수 출력
rawData_org.shape

(1000000, 34)

In [8]:
# 의사결정트리에 사용할 속성리스트
feature_columns_to_use = ['성별코드', '연령대코드(5세단위)', '신장(5Cm단위)', '체중(5Kg 단위)', '허리둘레', '시력(좌)', '시력(우)',
                    '청력(좌)', '청력(우)', '수축기혈압', '이완기혈압', '식전혈당(공복혈당)', '트리글리세라이드', 'HDL콜레스테롤',
                    'LDL콜레스테롤', '요단백', '혈청크레아티닌', '(혈청지오티)AST', '(혈청지오티)ALT', '감마지티피', '흡연상태', '음주여부']

rawData = rawData_org[feature_columns_to_use]

In [9]:
# 속성값이 NULL인 튜플 제외
rawData = rawData.dropna()

# 인덱스 재설정
rawData.reset_index(inplace=True, drop=True)

# 원본데이터의 튜플 수와 컬럼수 출력
rawData.shape

(990910, 22)

In [10]:
import random as rd  # 샘플링을 위한 random 패키지 임포트

# 1) 샘플링을 통한 수량 축소
# 원본데이터 중 50,000개를 튜플만 샘플링
# len() : 데이터프레임의 크기를 구하는 함수
sample_idx = rd.sample(range(0, len(rawData)), 50000)

# 인덱스 값 정렬(오름차순)
sample_idx.sort()

sample_idx

[14,
 15,
 43,
 58,
 60,
 68,
 89,
 143,
 201,
 213,
 223,
 308,
 330,
 331,
 333,
 337,
 352,
 361,
 376,
 427,
 433,
 481,
 482,
 512,
 534,
 570,
 613,
 614,
 631,
 640,
 673,
 683,
 705,
 721,
 750,
 799,
 815,
 818,
 826,
 843,
 844,
 845,
 867,
 877,
 878,
 970,
 998,
 1016,
 1041,
 1042,
 1043,
 1049,
 1060,
 1063,
 1068,
 1100,
 1107,
 1152,
 1160,
 1162,
 1166,
 1174,
 1176,
 1177,
 1178,
 1199,
 1201,
 1211,
 1223,
 1230,
 1240,
 1275,
 1284,
 1303,
 1307,
 1395,
 1410,
 1461,
 1496,
 1505,
 1567,
 1580,
 1593,
 1607,
 1611,
 1628,
 1652,
 1683,
 1690,
 1761,
 1763,
 1838,
 1839,
 1848,
 1851,
 1886,
 1893,
 1894,
 1928,
 1934,
 1937,
 1954,
 1962,
 2018,
 2039,
 2047,
 2066,
 2112,
 2118,
 2125,
 2202,
 2221,
 2256,
 2324,
 2327,
 2454,
 2461,
 2477,
 2486,
 2505,
 2512,
 2546,
 2558,
 2585,
 2626,
 2637,
 2646,
 2660,
 2681,
 2684,
 2703,
 2712,
 2721,
 2722,
 2741,
 2750,
 2756,
 2809,
 2817,
 2851,
 2941,
 2958,
 2969,
 2985,
 2994,
 3008,
 3119,
 3123,
 3127,
 3175,
 323

In [11]:
# 샘플링 된 인덱스로 구성된 샘플 데이터프레임 생성
rawData_sample = rawData.loc[sample_idx]

rawData_sample.reset_index(inplace=True, drop=True) #  인덱스 재설정

# 첫 10개의 행만 출력
rawData_sample.head(10)

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부
0,2,10,160.0,70.0,87.0,0.8,0.8,1.0,1.0,129.0,...,173.0,56.0,164.0,1.0,0.7,16.0,12.0,12.0,1.0,0.0
1,2,15,150.0,45.0,69.0,0.6,0.8,1.0,1.0,110.0,...,155.0,64.0,119.0,1.0,0.8,19.0,14.0,10.0,1.0,0.0
2,2,14,145.0,50.0,76.0,0.7,1.0,1.0,1.0,136.0,...,131.0,71.0,133.0,1.0,0.5,18.0,14.0,15.0,1.0,0.0
3,1,9,175.0,80.0,84.0,1.2,1.0,1.0,1.0,130.0,...,97.0,52.0,127.0,1.0,1.1,29.0,22.0,25.0,2.0,1.0
4,1,15,160.0,65.0,93.0,0.6,0.6,1.0,1.0,140.0,...,115.0,58.0,130.0,1.0,1.2,22.0,17.0,65.0,2.0,0.0
5,1,10,165.0,70.0,92.0,1.5,1.5,1.0,1.0,126.0,...,246.0,57.0,99.0,1.0,0.9,27.0,35.0,159.0,3.0,1.0
6,2,10,160.0,50.0,71.1,1.0,0.9,1.0,1.0,108.0,...,76.0,71.0,117.0,1.0,0.7,24.0,11.0,17.0,3.0,1.0
7,2,10,160.0,65.0,86.0,1.2,0.8,1.0,1.0,124.0,...,84.0,62.0,137.0,1.0,0.6,23.0,17.0,18.0,1.0,1.0
8,2,12,170.0,60.0,72.3,0.6,0.7,1.0,1.0,120.0,...,96.0,85.0,108.0,1.0,0.8,18.0,16.0,14.0,1.0,0.0
9,1,9,180.0,85.0,91.0,0.6,1.2,1.0,1.0,135.0,...,183.0,56.0,173.0,1.0,0.8,24.0,33.0,39.0,3.0,1.0


In [13]:
rawData_sample.shape

(50000, 22)

In [14]:
# nonnumeric 속성을 categrical 데이터로 변환
nonnumeric_columns = ['성별코드','음주여부']

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
for feature in nonnumeric_columns:
    rawData_sample[feature] = le.fit_transform(rawData_sample[feature])   

In [15]:
rawData_sample

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부
0,1,10,160.0,70.0,87.0,0.8,0.8,1.0,1.0,129.0,...,173.0,56.0,164.0,1.0,0.7,16.0,12.0,12.0,1.0,0
1,1,15,150.0,45.0,69.0,0.6,0.8,1.0,1.0,110.0,...,155.0,64.0,119.0,1.0,0.8,19.0,14.0,10.0,1.0,0
2,1,14,145.0,50.0,76.0,0.7,1.0,1.0,1.0,136.0,...,131.0,71.0,133.0,1.0,0.5,18.0,14.0,15.0,1.0,0
3,0,9,175.0,80.0,84.0,1.2,1.0,1.0,1.0,130.0,...,97.0,52.0,127.0,1.0,1.1,29.0,22.0,25.0,2.0,1
4,0,15,160.0,65.0,93.0,0.6,0.6,1.0,1.0,140.0,...,115.0,58.0,130.0,1.0,1.2,22.0,17.0,65.0,2.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,1,9,155.0,50.0,66.0,0.9,0.8,1.0,1.0,99.0,...,43.0,82.0,88.0,1.0,0.6,23.0,18.0,13.0,1.0,0
49996,0,13,165.0,60.0,86.0,0.8,1.2,1.0,1.0,120.0,...,126.0,54.0,146.0,4.0,1.5,25.0,42.0,30.0,1.0,1
49997,0,12,160.0,55.0,76.0,0.7,0.7,1.0,1.0,120.0,...,73.0,59.0,66.0,1.0,1.2,25.0,29.0,13.0,2.0,0
49998,1,14,150.0,65.0,92.0,0.4,0.4,1.0,1.0,130.0,...,62.0,55.0,92.0,1.0,0.7,20.0,14.0,16.0,1.0,0


In [16]:
# 소스 데이터프레임에서 분류(classification)을 위한 속성 집합
X = rawData_sample.loc[:, feature_columns_to_use[:-1]]  
y = rawData_sample.loc[:, '음주여부']  # 분류 클래스 (class)
  
from sklearn.model_selection import train_test_split  # 분석모형 선택에 관련된 모듈
    
# 자동으로 데이터셋을 트레이닝셋과 테스트셋으로 분리해주는 함수로
# 트레이닝셋과 데이터셋의 비율을 7:3으로 세팅함
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [17]:
# RMSLE 계산하는 사용자정의 함수
from sklearn.metrics import make_scorer

def rmsle(predicted_values, actual_values) :
    # 넘파이로 배열 형태로 변환
    predicted_values = np.array(predicted_values)
    actual_values = np.array(actual_values)
    
    # 예측값과 실제값에 1을 더하고 로그를 씌움
    log_predict = np.log(predicted_values + 1)
    log_actual = np.log(actual_values + 1)
    
    # 위에서 계산한 예측값에서 실제값을 빼주고 제곱을 함
    difference = log_predict - log_actual
    # difference = (log_predict - log_actual) ** 2
    difference = np.square(difference)
    
    # 평균값 구함
    mean_difference = difference.mean()
    
    # 다시 루트를 씌움
    score = np.sqrt(mean_difference)
    
    return score

rmsle_scorer = make_scorer(rmsle)
rmsle_scorer

make_scorer(rmsle)

In [18]:
# Scikit-Learn 패키지 : 머신 러닝 교육 및 실무를 위한 패키지로 샘플 데이터셋,
# 다양한 기계학습 기법에 대한 함수 등을 포함하고 있음
from sklearn.ensemble import RandomForestClassifier  # 랜덤포리스트 기법에 관련된 모듈

# DecisionTreeClassifier() : 의사결정트리를 생성하는 함수
random_forest = RandomForestClassifier(n_estimators=20, random_state=0)
random_forest

RandomForestClassifier(n_estimators=20, random_state=0)

In [19]:
# KFold 교차검증
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

k_fold = KFold(n_splits=10, shuffle=True, random_state=0)

# 트레이닝셋에 대한 KFold 교차검증 수
%time score = cross_val_score(random_forest, X_train, y_train, cv=k_fold, scoring=rmsle_scorer)
score = score.mean()

# 0에 근접할수록 좋은 데이터
print("Score= {0:.5f}".format(score))

Wall time: 13.7 s
Score= 0.37086


In [25]:
# fit() : 트레이닝 데이터셋을 대상으로 의사결정트리 학습 진행
random_forest.fit(X_train, y_train)

# tree.predict() 함수를 활용하여 의사결정트리를 대상으로 테스트셋을 예측
y_pred_tr = random_forest.predict(X_test)

y_pred_tr_prob = random_forest.predict_proba(X_test)

In [26]:
y_pred_tr_prob

array([[0.35, 0.65],
       [0.2 , 0.8 ],
       [0.3 , 0.7 ],
       ...,
       [0.05, 0.95],
       [0.95, 0.05],
       [0.9 , 0.1 ]])

In [27]:
# 점수 출력
print("Train Set Score1 : {:.2f}".format(random_forest.score(X_train, y_train)))
print("Test Set Score1 : {:.2f}".format(random_forest.score(X_test, y_test)))

Train Set Score1 : 1.00
Test Set Score1 : 0.72


In [28]:
from sklearn.metrics import accuracy_score  # 분류 정확도(classification accuracy)를 계산하는 모듈

# accuracy_score() 함수를 활용하여 테스트셋의 실제 클래스와 예측된 클래스 간 정확도 측정
print('Accuracy: %.2f' % accuracy_score(y_test, y_pred_tr))

Accuracy: 0.72


In [29]:
# 속성(feature) 별 중요도를 저장하는 데이터프레임 생성 
sel_feature = pd.DataFrame({'중요도' : random_forest.feature_importances_}, index = feature_columns_to_use[:-1])

# 중요도의 내림차순으로 정렬
sel_feature.sort_values(by='중요도', ascending=False)

,중요도
감마지티피,0.084531
연령대코드(5세단위),0.076422
HDL콜레스테롤,0.068833
트리글리세라이드,0.063589
LDL콜레스테롤,0.063110
흡연상태,0.059570
신장(5Cm단위),0.057950
허리둘레,0.055681
식전혈당(공복혈당),0.055559
(혈청지오티)ALT,0.055107


In [30]:
### 중요도 순으로 상위 10개 속성 선택

In [32]:
appData_anal_comp = pd.DataFrame(data={'actual':y_test, 'pred':y_pred_tr}, columns = {'actual','pred'})
appData_anal_comp

,pred,actual
11841,1,0
19602,1,1
45519,1,1
25747,0,0
42642,0,0
...,...,...
38344,0,0
49984,0,0
32624,1,1
46437,0,0


In [39]:
X_test['음주여부']=y_test
X_test['음주여부예측'] = y_pred_tr

X_test['음주확률'] = y_pred_tr_prob[:,1]
X_test

<ipython-input-39-19aa5ac9380b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['음주여부']=y_test
<ipython-input-39-19aa5ac9380b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['음주여부예측'] = y_pred_tr
<ipython-input-39-19aa5ac9380b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returni

,성별코드,연령대코드(5세단위),신장(5Cm단위),체중(5Kg 단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,LDL콜레스테롤,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,흡연상태,음주여부,음주여부예측,음주확률
11841,0,10,170.0,80.0,94.0,0.8,1.0,1.0,1.0,110.0,...,116.0,1.0,1.0,26.0,20.0,30.0,3.0,0,1,0.65
19602,0,10,170.0,85.0,102.0,0.9,0.9,1.0,1.0,130.0,...,106.0,1.0,0.9,67.0,119.0,216.0,3.0,1,1,0.80
45519,0,13,160.0,40.0,76.0,1.2,1.2,1.0,1.0,146.0,...,73.0,1.0,0.8,107.0,50.0,328.0,2.0,1,1,0.70
25747,1,12,150.0,50.0,83.0,0.4,0.4,1.0,1.0,145.0,...,35.0,1.0,0.7,32.0,24.0,55.0,3.0,0,0,0.50
42642,0,7,170.0,60.0,78.0,1.2,1.2,1.0,1.0,118.0,...,114.0,1.0,1.0,18.0,26.0,22.0,1.0,0,0,0.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38344,1,11,150.0,40.0,66.0,1.0,1.0,1.0,1.0,110.0,...,88.0,1.0,0.5,26.0,20.0,26.0,1.0,0,0,0.45
49984,0,17,155.0,55.0,87.0,0.7,1.0,1.0,1.0,130.0,...,62.0,1.0,1.4,25.0,20.0,27.0,2.0,0,0,0.10
32624,0,10,175.0,80.0,92.0,1.0,1.0,1.0,1.0,120.0,...,80.0,1.0,0.7,20.0,22.0,46.0,3.0,1,1,0.95
46437,1,17,150.0,60.0,102.1,0.5,0.5,2.0,1.0,124.0,...,55.0,1.0,0.7,25.0,14.0,14.0,1.0,0,0,0.05
